# Solution

The solution is simple, we need to use the ``queryCarsHistory`` method and check the mileage of the last car compared to the proposed one. It can be done using two ``if`` statements as below.

In [ ]:
    @Transaction()
    public Car addCarRepair(final Context ctx, final String vin, final Integer mileage, final String repairName) {
        ChaincodeStub stub = ctx.getStub();

        Car car = new Car(mileage, repairName);

        Car[] carHistory = this.queryCarsHistory(ctx, vin);
        if (carHistory != null && carHistory.length > 0) {
            Car previousRepair = carHistory[carHistory.length - 1];
            if (Integer.parseInt(previousRepair.getMileage()) <= mileage) {
              String carState = genson.serialize(car);
              stub.putStringState(vin, carState);
            } else {
              return car;
            }
        } else {
            String carState = genson.serialize(car);
            stub.putStringState(vin, carState);
        }
        return car;
    }

## Build and deploy 

You should build it in the same way as the previous smart contract:
``./gradlew clean build shadowJar``. Keep in mind that the deployment should be done using a new chaincode name like ``newcarcc``:

``peer chaincode install -l java -n newcarcc -v v1 -p <path>``


``peer chaincode instantiate -o orderer.codete.com:7050 -C channelone -n newcarcc -v v1 -c '{"Args":[]}' -P 'OR ("Org1MSP.member")'``

## Testing

Test it by adding a new block:

``peer chaincode invoke -n newcarcc -c '{"Args":["addCarRepair", "12345", "3000", "WheelChange"]}' -C channelone``


Next try to add a block with smaller mileage:

``peer chaincode invoke -n newcarcc -c '{"Args":["addCarRepair", "12345", "200", "WheelChange"]}' -C channelone``

And next add a block with mileage that is greater than the first one:
``peer chaincode invoke -n newcarcc -c '{"Args":["addCarRepair", "12345", "5000", "WheelChange"]}' -C channelone``


Check the car repairs history:
``peer chaincode invoke -n newcarcc -c '{"Args":["queryCarsHistory", "12345"]}' -C channelone``